# Week 10 — Kan-Do Calculus with Radon–Nikodym Ratios
We compare conditioning vs intervention using a simple RN reweighting idea.


In [ ]:
import torch
import math
import matplotlib.pyplot as plt

torch.manual_seed(0)

# Base model: X ~ Normal(0,1); Y = X + noise
N = 20000
X = torch.randn(N)
Y = X + 0.5 * torch.randn(N)

# Conditioning: approximate p(Y | X=1) by filtering nearby samples
mask = (X > 0.9) & (X < 1.1)
Y_cond = Y[mask]

# Intervention: do(X=1). Use RN ratio to reweight p(Y|X) to p(Y|do(X=1))
# Here we model p(X) ~ Normal(0,1) and p(X|do=1) as a delta at 1.
# Approximate RN ratio with a narrow Normal around 1.
sigma = 0.05
p_x = torch.distributions.Normal(0.0, 1.0)
p_do = torch.distributions.Normal(1.0, sigma)
rn = torch.exp(p_do.log_prob(X) - p_x.log_prob(X))
rn = rn / rn.mean()

# Reweight Y samples
weights = rn
Y_do_mean = (weights * Y).mean().item()
Y_do_var = (weights * (Y - Y_do_mean) ** 2).mean().item()

print('E[Y | X≈1] (conditioning):', float(Y_cond.mean()))
print('E[Y | do(X=1)] (RN reweight):', round(Y_do_mean, 3))

plt.figure(figsize=(6,4))
plt.hist(Y.numpy(), bins=50, alpha=0.4, label='Y (observed)')
plt.hist(Y_cond.numpy(), bins=50, alpha=0.6, label='Y | X≈1')
plt.axvline(Y_do_mean, color='k', linestyle='--', label='E[Y | do(X=1)]')
plt.legend()
plt.title('Conditioning vs intervention')
plt.tight_layout()
plt.show()
